In [ ]:
import os
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from matplotlib import pyplot as plt

In [ ]:
# Função para aplicar o detector de bordas (Sobel)
def edge_detection(image):
    edges = cv2.Sobel(image, cv2.CV_64F, 1, 1, ksize=5)
    edges = cv2.convertScaleAbs(edges)
    return edges

# Função para binarização local (Niblack)
def local_binarization(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binarized_image = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return binarized_image

# Função para binarização global (Otsu)
def global_binarization(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binarized_image = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return binarized_image

# Função para detecção de bordas por Marr-Hildreth
def marr_hildreth_edge_detection(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Laplacian(gray, cv2.CV_64F)
    edges = cv2.convertScaleAbs(edges)
    return edges

# Função para aplicar o algoritmo DBSCAN
def dbscan_clustering(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    pixels = np.column_stack(np.where(gray > 0))
    dbscan = DBSCAN(eps=5, min_samples=10)
    labels = dbscan.fit_predict(pixels)
    mask = np.zeros_like(gray, dtype=np.uint8)
    for label in np.unique(labels):
        cluster_mask = (labels == label)
        mask[pixels[cluster_mask][:, 0], pixels[cluster_mask][:, 1]] = 255
    return mask

# Função para escolher o foreground e o background automaticamente
def auto_foreground_background(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    foreground = cv2.bitwise_and(image, image, mask=binary)
    background = cv2.bitwise_and(image, image, mask=cv2.bitwise_not(binary))
    return foreground, background

In [ ]:
# Caminhos para as pastas de entrada (imagens) e saída (resultados)
input_folder = 'ultrassound\\cropped'
output_folder = 'result'  # Altere para o caminho desejado

# Obter o caminho completo para a pasta de entrada
input_folder_full = os.path.join(os.getcwd(), input_folder)

# Obter o caminho completo para a pasta de saída
output_folder_full = os.path.join(os.getcwd(), output_folder)

# Verificar se a pasta de entrada existe
if not os.path.exists(input_folder_full):
    print(f"A pasta de entrada '{input_folder_full}' não foi encontrada.")
    exit()

In [ ]:
# Criar a pasta de saída se não existir
os.makedirs(output_folder_full, exist_ok=True)

In [ ]:
# Lista de arquivos na pasta de entrada
image_files = os.listdir(input_folder_full)

In [ ]:
# Loop sobre todas as imagens na pasta de entrada
for image_file in image_files:
    # Construir o caminho completo da imagem de entrada
    image_path = os.path.join(input_folder_full, image_file)

    # Tente carregar a imagem
    image = cv2.imread(image_path)

    # Verifique se a imagem foi carregada com sucesso
    if image is None:
        print(f'Erro ao carregar a imagem: {image_path}')
        continue
        
    """I. Aplicar K-means e Detector de Borda"""
    pixels = image.reshape((-1, 3))
    k = 5
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(pixels)
    centers = np.uint8(kmeans.cluster_centers_)
    segmented_image = centers[kmeans.labels_].reshape(image.shape)
    edges_kmeans = edge_detection(segmented_image)

    # Salvar os resultados da questão I
    output_path_i = os.path.join(output_folder, f'result_i_{os.path.splitext(image_file)[0]}.png')
    cv2.imwrite(output_path_i, edges_kmeans)

    """II. Implementar duas segmentações utilizando os algoritmos de binarização local e global:"""
    binarized_local = local_binarization(image)
    binarized_global = global_binarization(image)
    edges_local = edge_detection(binarized_local)
    edges_global = edge_detection(binarized_global)

    # Salvar os resultados da questão II
    output_path_ii_local = os.path.join(output_folder, f'result_ii_local_{os.path.splitext(image_file)[0]}.png')
    output_path_ii_global = os.path.join(output_folder, f'result_ii_global_{os.path.splitext(image_file)[0]}.png')
    cv2.imwrite(output_path_ii_local, edges_local)
    cv2.imwrite(output_path_ii_global, edges_global)

    """III. Realizar a detecção de bordas utilizando o método de Marr-Hildreth:"""
    edges_marr_hildreth = marr_hildreth_edge_detection(image)

    # Salvar os resultados da questão III
    output_path_iii = os.path.join(output_folder, f'result_iii_{os.path.splitext(image_file)[0]}.png')
    cv2.imwrite(output_path_iii, edges_marr_hildreth)

    """IV. Aplicar outro algoritmo de agrupamento e utilizar um detector de borda:"""
    clustered_dbscan = dbscan_clustering(image)
    edges_dbscan = edge_detection(clustered_dbscan)

    # Salvar os resultados da questão IV
    output_path_iv_clustered = os.path.join(output_folder, f'result_iv_clustered_{os.path.splitext(image_file)[0]}.png')
    output_path_iv_edges = os.path.join(output_folder, f'result_iv_edges_{os.path.splitext(image_file)[0]}.png')
    cv2.imwrite(output_path_iv_clustered, clustered_dbscan)
    cv2.imwrite(output_path_iv_edges, edges_dbscan)

    """
    Este trecho do código foi mantido para a questão 2,
    mas ajuste conforme necessário para a lógica completa.
    """
    foreground, background = auto_foreground_background(image)
    output_path_v_foreground = os.path.join(output_folder, f'result_v_foreground_{os.path.splitext(image_file)[0]}.png')
    output_path_v_background = os.path.join(output_folder, f'result_v_background_{os.path.splitext(image_file)[0]}.png')
    cv2.imwrite(output_path_v_foreground, foreground)
    cv2.imwrite(output_path_v_background, background)